In [39]:
import open3d as o3d
import numpy as np

# Create a mesh of a sphere
mesh = o3d.geometry.TriangleMesh.create_sphere(radius=1.0)
mesh.compute_vertex_normals()

# Sample points uniformly from the sphere mesh
pcd_uniform = mesh.sample_points_poisson_disk(number_of_points=4096)

# Convert Open3D PointCloud to NumPy array for manipulation
points = np.asarray(pcd_uniform.points)

# std_dev = 0.1
# noise = np.random.normal(0, std_dev, points.shape)
# points += noise

# Create a new point cloud with non-uniform sampling and noise
pcd_non_uniform_noisy = o3d.geometry.PointCloud()
pcd_non_uniform_noisy.points = o3d.utility.Vector3dVector(points)

# Visualize the non-uniformly sampled point cloud with noise
o3d.visualization.draw_geometries([pcd_non_uniform_noisy])


In [40]:
import open3d as o3d
import numpy as np

# Create a mesh of a sphere
mesh = o3d.geometry.TriangleMesh.create_sphere(radius=1.0)
mesh.compute_vertex_normals()

# Sample points uniformly from the sphere mesh
pcd_uniform = mesh.sample_points_poisson_disk(number_of_points=4096)

# Convert Open3D PointCloud to NumPy array for manipulation
points = np.asarray(pcd_uniform.points)

# Apply non-uniform density based on distance from origin
# Calculate the distance of each point from the origin
distances = np.linalg.norm(points, axis=1)

# Determine a threshold distance for higher density
threshold_distance = 0.5

# Indices of points within the higher density region
high_density_indices = np.where(distances < threshold_distance)[0]

# Duplicate points within the higher density region
# Here we're simply concatenating the high-density points to the original points
# You could also apply a more complex function to vary the number of duplicates based on distance or other criteria
points = np.concatenate((points, points[high_density_indices]), axis=0)

# Optionally, you can add noise to the entire point set again
# std_dev = 0.1
# noise = np.random.normal(0, std_dev, points.shape)
# points += noise

# Create a new point cloud with the manipulated point distribution
pcd_non_uniform_noisy = o3d.geometry.PointCloud()
pcd_non_uniform_noisy.points = o3d.utility.Vector3dVector(points)

# Visualize the non-uniformly sampled point cloud with noise
o3d.visualization.draw_geometries([pcd_non_uniform_noisy])


In [46]:
import open3d as o3d
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset, WeightedRandomSampler

# Create a mesh of a sphere
mesh = o3d.geometry.TriangleMesh.create_sphere(radius=1.0)
mesh.compute_vertex_normals()

# Sample points uniformly from the sphere mesh to create an initial point cloud
pcd_uniform = mesh.sample_points_poisson_disk(number_of_points=8192)

# Convert Open3D PointCloud to NumPy array for manipulation
points = np.asarray(pcd_uniform.points)

# Define a simple criterion for non-uniformity based on the z-coordinate
# In this case, points with higher z-coordinate values will have higher sampling probability
weights = points[:, 2]  # Using the z-coordinate as weight
weights = weights - np.min(weights) + 1  # Making sure all weights are positive and not zero

# Convert points and weights to PyTorch tensors
points_tensor = torch.tensor(points, dtype=torch.float32)
weights_tensor = torch.tensor(weights, dtype=torch.float32)

# Create a WeightedRandomSampler to sample points with the given weights
num_samples_to_select = 4096  # You can adjust this number
sampler = WeightedRandomSampler(weights_tensor, num_samples=num_samples_to_select, replacement=True)

# Create a DataLoader with the sampler
points_dataset = TensorDataset(points_tensor)
points_loader = DataLoader(points_dataset, batch_size=1, sampler=sampler)

# Sample points based on the defined weights
sampled_points_list = []
for point in points_loader:
    sampled_points_list.append(point[0].numpy())

# Convert the list of sampled points to a NumPy array
sampled_points = np.vstack(sampled_points_list)

# Create a new point cloud with the non-uniformly sampled points
pcd_non_uniform = o3d.geometry.PointCloud()
pcd_non_uniform.points = o3d.utility.Vector3dVector(sampled_points)

# Visualize the non-uniformly sampled point cloud
o3d.visualization.draw_geometries([pcd_non_uniform])


In [47]:
# inp_pcd = o3d.geometry.PointCloud()
# inp_pcd.points = o3d.utility.Vector3dVector(points)
down_samp = pcd_non_uniform.voxel_down_sample( voxel_size=0.1)
print(np.array(down_samp.points).shape)
# inp_pcd.paint_uniform_color([1, 0, 0])  # Sets the point cloud color to red
# down_samp.paint_uniform_color([0, 1, 0])  # Sets the point cloud color to green
o3d.visualization.draw_geometries([down_samp])

(1321, 3)
